In [1]:
import pandas as pd
import glob
import tensorflow as tf
from tensorflow.data import Dataset

2021-10-18 21:02:21.351187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 21:02:21.351228: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path =r'../data/datasets_by_year/'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    df = pd.read_csv(filename).drop(columns=["Unnamed: 0", "individual-local-identifier"])
    if df.shape > (100,):
        dfs.append(df)

In [3]:
for df in dfs:
    print(df.shape)

(142, 6)
(129, 6)
(111, 6)
(188, 6)
(168, 6)
(238, 6)
(128, 6)
(271, 6)
(184, 6)
(241, 6)
(140, 6)
(186, 6)
(156, 6)
(415, 6)
(201, 6)
(138, 6)
(299, 6)
(249, 6)
(382, 6)
(316, 6)
(171, 6)
(119, 6)
(191, 6)
(245, 6)
(107, 6)
(229, 6)
(234, 6)
(345, 6)
(201, 6)
(148, 6)
(234, 6)
(122, 6)
(278, 6)
(284, 6)
(316, 6)
(170, 6)
(370, 6)
(263, 6)
(127, 6)
(199, 6)
(282, 6)
(354, 6)
(148, 6)
(453, 6)
(275, 6)
(208, 6)
(392, 6)
(219, 6)
(178, 6)
(347, 6)
(278, 6)
(229, 6)
(150, 6)
(139, 6)
(352, 6)
(321, 6)


In [4]:
def split_data(df):

    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    num_features = df.shape[1]
    
    return train_df, val_df,test_df

In [5]:
split_data_result = [split_data(df) for df in dfs]

In [6]:
def data_windowing(train_df,val_df,test_df):
    n_points = 20
    window_length = n_points+ 1 
    
    window_train = tf.keras.utils.timeseries_dataset_from_array(
    train_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None)
    
    window_test = tf.keras.utils.timeseries_dataset_from_array(
    test_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None)
        
    window_val = tf.keras.utils.timeseries_dataset_from_array(
    val_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None)
    
    return window_train, window_test, window_val

In [7]:
def data_map(elem):
    n_points = 20
    window_length = n_points+ 1 
    X,y = elem[:,:-1,:],elem[:,-1:,0:2]
    X.set_shape([None, n_points, None])
    y.set_shape([None, window_length - n_points, None])
    
    return X,y

In [8]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(units=2)
])

2021-10-18 21:02:25.792757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-18 21:02:25.792797: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-18 21:02:25.792811: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-R3AGIN3): /proc/driver/nvidia/version does not exist
2021-10-18 21:02:25.792986: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
lstm_model.compile(loss=tf.losses.MeanSquaredError(),optimizer=tf.optimizers.Adam(),metrics=[tf.metrics.MeanAbsoluteError()])

In [10]:
for i, (train_df,val_df,test_df) in enumerate(split_data_result):
    
    print(i,len(split_data_result))
    
    window_train,window_val,window_test = data_windowing(train_df,val_df,test_df)
    train = window_train.map(data_map)
    test = window_test.map(data_map)
    val = window_val.map(data_map)
    
    if i == 1:
        print(train)
        print(test)
        print(val)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                    patience=2,
                                                    mode='min')
    
    history = lstm_model.fit(train, epochs=20,
                      validation_data=val,
                      callbacks=[early_stopping])

0 56
Epoch 1/20


2021-10-18 21:02:26.928184: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3/3 [==============================] - 1s 40ms/step - loss: 7301.4448 - mean_absolute_error: 73.1465
Epoch 2/20
3/3 [==============================] - 0s 39ms/step - loss: 7271.7397 - mean_absolute_error: 72.9608
Epoch 3/20
3/3 [==============================] - 0s 34ms/step - loss: 7247.7026 - mean_absolute_error: 72.8278
Epoch 4/20
3/3 [==============================] - 0s 33ms/step - loss: 7225.1196 - mean_absolute_error: 72.7374
Epoch 5/20
3/3 [==============================] - 0s 32ms/step - loss: 7184.3535 - mean_absolute_error: 72.6358
Epoch 6/20
3/3 [==============================] - 0s 33ms/step - loss: 7146.8188 - mean_absolute_error: 72.5378
Epoch 7/20
3/3 [==============================] - 0s 34ms/step - loss: 7124.3379 - mean_absolute_error: 72.4500
Epoch 8/20
3/3 [==============================] - 0s 32ms/step - loss: 7096.0459 - mean_absolute_error: 72.3338
Epoch 9/20
3/3 [==============================] - 0s 35ms/step - loss: 7058.0552 - mean_absolute_error: 72.1799
Epo

4/4 [==============================] - 0s 26ms/step - loss: 6127.6509 - mean_absolute_error: 66.2844
Epoch 12/20
4/4 [==============================] - 0s 24ms/step - loss: 6109.6094 - mean_absolute_error: 66.1414
Epoch 13/20
4/4 [==============================] - 0s 25ms/step - loss: 6093.7383 - mean_absolute_error: 66.0247
Epoch 14/20
4/4 [==============================] - 0s 24ms/step - loss: 6076.2847 - mean_absolute_error: 65.8961
Epoch 15/20
4/4 [==============================] - 0s 24ms/step - loss: 6058.0625 - mean_absolute_error: 65.7623
Epoch 16/20
4/4 [==============================] - 0s 24ms/step - loss: 6042.6597 - mean_absolute_error: 65.6491
Epoch 17/20
4/4 [==============================] - 0s 24ms/step - loss: 6027.9277 - mean_absolute_error: 65.5407
Epoch 18/20
4/4 [==============================] - 0s 24ms/step - loss: 6013.2759 - mean_absolute_error: 65.4327
Epoch 19/20
4/4 [==============================] - 0s 24ms/step - loss: 5998.7031 - mean_absolute_error: 65.

3/3 [==============================] - 0s 31ms/step - loss: 5379.0317 - mean_absolute_error: 60.5422
Epoch 14/20
3/3 [==============================] - 0s 32ms/step - loss: 5370.2725 - mean_absolute_error: 60.4764
Epoch 15/20
3/3 [==============================] - ETA: 0s - loss: 5537.8047 - mean_absolute_error: 62.464 - 0s 30ms/step - loss: 5361.5303 - mean_absolute_error: 60.4108
Epoch 16/20
3/3 [==============================] - 0s 33ms/step - loss: 5352.8042 - mean_absolute_error: 60.3452
Epoch 17/20
3/3 [==============================] - 0s 31ms/step - loss: 5344.0938 - mean_absolute_error: 60.2797
Epoch 18/20
3/3 [==============================] - 0s 30ms/step - loss: 5335.3989 - mean_absolute_error: 60.2143
Epoch 19/20
3/3 [==============================] - 0s 30ms/step - loss: 5326.7183 - mean_absolute_error: 60.1489
Epoch 20/20
3/3 [==============================] - 0s 31ms/step - loss: 5318.0527 - mean_absolute_error: 60.0837
7 56
Epoch 1/20
6/6 [=============================

...

10/10 [==============================] - 0s 15ms/step - loss: 244.4686 - mean_absolute_error: 11.4440 - val_loss: 229.6290 - val_mean_absolute_error: 10.7516
Epoch 4/20
10/10 [==============================] - 0s 15ms/step - loss: 241.6191 - mean_absolute_error: 11.3772 - val_loss: 226.8510 - val_mean_absolute_error: 10.6805
Epoch 5/20
10/10 [==============================] - 0s 15ms/step - loss: 238.7680 - mean_absolute_error: 11.3087 - val_loss: 224.0852 - val_mean_absolute_error: 10.6083
Epoch 6/20
10/10 [==============================] - 0s 16ms/step - loss: 235.9329 - mean_absolute_error: 11.2398 - val_loss: 221.3400 - val_mean_absolute_error: 10.5358
Epoch 7/20
10/10 [==============================] - 0s 17ms/step - loss: 233.1199 - mean_absolute_error: 11.1709 - val_loss: 218.6184 - val_mean_absolute_error: 10.4635
Epoch 8/20
10/10 [==============================] - 0s 15ms/step - loss: 230.3310 - mean_absolute_error: 11.1022 - val_loss: 215.9214 - val_mean_absolute_error: 10.39

Epoch 12/20
4/4 [==============================] - 0s 26ms/step - loss: 191.2348 - mean_absolute_error: 10.8921 - val_loss: 213.7174 - val_mean_absolute_error: 12.1696
Epoch 13/20
4/4 [==============================] - 0s 24ms/step - loss: 190.2637 - mean_absolute_error: 10.8644 - val_loss: 214.1424 - val_mean_absolute_error: 12.2006
Epoch 14/20
4/4 [==============================] - 0s 25ms/step - loss: 189.2971 - mean_absolute_error: 10.8367 - val_loss: 214.5677 - val_mean_absolute_error: 12.2314
Epoch 15/20
4/4 [==============================] - 0s 26ms/step - loss: 188.3348 - mean_absolute_error: 10.8091 - val_loss: 214.9927 - val_mean_absolute_error: 12.2622
Epoch 16/20
4/4 [==============================] - 0s 25ms/step - loss: 187.3769 - mean_absolute_error: 10.7815 - val_loss: 215.4178 - val_mean_absolute_error: 12.2928
Epoch 17/20
4/4 [==============================] - 0s 25ms/step - loss: 186.4232 - mean_absolute_error: 10.7540 - val_loss: 215.8427 - val_mean_absolute_error: 

4/4 [==============================] - 0s 26ms/step - loss: 133.3126 - mean_absolute_error: 9.2137
Epoch 2/20
4/4 [==============================] - 0s 24ms/step - loss: 132.6627 - mean_absolute_error: 9.1928
Epoch 3/20
4/4 [==============================] - 0s 24ms/step - loss: 131.9419 - mean_absolute_error: 9.1631
Epoch 4/20
4/4 [==============================] - 0s 24ms/step - loss: 131.1762 - mean_absolute_error: 9.1276
Epoch 5/20
4/4 [==============================] - 0s 23ms/step - loss: 130.3833 - mean_absolute_error: 9.0883
Epoch 6/20
4/4 [==============================] - 0s 23ms/step - loss: 129.5748 - mean_absolute_error: 9.0466
Epoch 7/20
4/4 [==============================] - 0s 25ms/step - loss: 128.7583 - mean_absolute_error: 9.0035
Epoch 8/20
4/4 [==============================] - 0s 27ms/step - loss: 127.9391 - mean_absolute_error: 8.9594
Epoch 9/20
4/4 [==============================] - 0s 26ms/step - loss: 127.1204 - mean_absolute_error: 8.9149
Epoch 10/20
4/4 [====

6/6 [==============================] - 0s 22ms/step - loss: 93.9586 - mean_absolute_error: 7.2120 - val_loss: 50.8613 - val_mean_absolute_error: 5.1377
Epoch 18/20
6/6 [==============================] - 0s 22ms/step - loss: 93.1466 - mean_absolute_error: 7.1740 - val_loss: 50.6105 - val_mean_absolute_error: 5.0959
Epoch 19/20
6/6 [==============================] - 0s 20ms/step - loss: 92.3417 - mean_absolute_error: 7.1362 - val_loss: 50.3676 - val_mean_absolute_error: 5.0545
Epoch 20/20
6/6 [==============================] - 0s 19ms/step - loss: 91.5440 - mean_absolute_error: 7.0986 - val_loss: 50.1327 - val_mean_absolute_error: 5.0180
51 56
Epoch 1/20
5/5 [==============================] - 0s 23ms/step - loss: 101.8668 - mean_absolute_error: 7.3092 - val_loss: 103.2629 - val_mean_absolute_error: 8.5009
Epoch 2/20
5/5 [==============================] - 0s 21ms/step - loss: 101.1485 - mean_absolute_error: 7.2816 - val_loss: 102.6819 - val_mean_absolute_error: 8.4759
Epoch 3/20
5/5 [====

3/3 [==============================] - 0s 31ms/step - loss: 76.0902 - mean_absolute_error: 6.0169
Epoch 19/20
3/3 [==============================] - 0s 31ms/step - loss: 75.7687 - mean_absolute_error: 5.9982
Epoch 20/20
3/3 [==============================] - 0s 30ms/step - loss: 75.4490 - mean_absolute_error: 5.9795
54 56
Epoch 1/20
8/8 [==============================] - 0s 18ms/step - loss: 448.6717 - mean_absolute_error: 20.3435 - val_loss: 46.3935 - val_mean_absolute_error: 5.6773
Epoch 2/20
8/8 [==============================] - 0s 17ms/step - loss: 436.7547 - mean_absolute_error: 20.1083 - val_loss: 43.4021 - val_mean_absolute_error: 5.4327
Epoch 3/20
8/8 [==============================] - 0s 16ms/step - loss: 419.6725 - mean_absolute_error: 19.7569 - val_loss: 40.5508 - val_mean_absolute_error: 5.1656
Epoch 4/20
8/8 [==============================] - 0s 16ms/step - loss: 401.5166 - mean_absolute_error: 19.3712 - val_loss: 38.2058 - val_mean_absolute_error: 4.9011
Epoch 5/20
8/8 [